name: keras_retraining.ipynb

This notebook was used to prototype code where we can change how the different layers of the MPRA-DragoNN model are treated during training. This involves initializing randomly, initializing from a starting point and commencing training, and initializing from a starting point and freezing training. 

---

https://keras.io/guides/transfer_learning/

In [ ]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/Shareddrives/NRC_Amii_Agronomics_Project/nrc-ml-plant-genomics/'

In [ ]:
import keras
import warnings, logging
import json
import numpy as np
import pandas as pd

from models.base_model import BaseModel
from keras.models import Sequential
from keras.layers import Input, Dense, Conv1D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.models import model_from_json
import tensorflow as tf

from sklearn.metrics import r2_score
from scipy.stats import spearmanr # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html

warnings.filterwarnings('ignore')
logging.disable(1000)

Using TensorFlow backend.


In [ ]:
# load in model as it is

with open('models/model.json', 'r') as json_file:
    json_savedModel = json_file.read()
    
pretrained_model = model_from_json(json_savedModel)
pretrained_model.load_weights('models/pretrained.hdf5')

In [ ]:
pretrained_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batch_normalization_1 (Batch (None, 141, 120)          480       
_________________________________________________________________
dropout_1 (Dropout)          (None, 141, 120)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batch_normalization_2 (Batch (None, 137, 120)          480       
_________________________________________________________________
dropout_2 (Dropout)          (None, 137, 120)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# one-hot encode sequence
mapping = {"A":[1, 0, 0, 0], "T":[0, 0, 0, 1], "C":[0, 1, 0, 0], "G":[0, 0, 1, 0]}  # cross referenced with kipoi data loader

def get_ohe(sequence):  # creates (145,4) representation of sequence
    return np.array([mapping[nt] for nt in sequence])

# def create_model(input_sequence_length, number_of_outputs):
#     model = Sequential()

#     model.add(Conv1D(120, 5, activation='relu', input_shape=(input_sequence_length, 4), name="1DConv_1", trainable=False))
#     model.add(BatchNormalization(name="batchNorm1", trainable=False))
#     model.add(Dropout(0.1, name="drop1"))

#     model.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=False))
#     model.add(BatchNormalization(name="batchNorm2", trainable=False))
#     model.add(Dropout(0.1, name="drop2"))

#     model.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=False))
#     model.add(BatchNormalization(name="batchNorm3", trainable=False))
#     model.add(Dropout(0.1, name="drop3"))

#     model.add(Flatten(name="flat"))
#     model.add(Dense(number_of_outputs, activation='linear', name="dense1"))
    
#     return model

---
### Chloroplast w/ Extra Layer

In [ ]:
def create_model(input_sequence_length, number_of_outputs):
    model = Sequential()

    # first conv layer
    model.add(Conv1D(120, 5, activation='relu', input_shape=(input_sequence_length, 4), name="1DConv_1", trainable=False))
    model.add(BatchNormalization(name="batchNorm1", trainable=False))
    model.add(Dropout(0.1, name="drop1"))

    # second conv layer
    model.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=False))
    model.add(BatchNormalization(name="batchNorm2", trainable=False))
    model.add(Dropout(0.1, name="drop2"))

    # third conv layer
    model.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=False))
    model.add(BatchNormalization(name="batchNorm3", trainable=False))
    model.add(Dropout(0.1, name="drop3"))

    # flatten and previous dense output
    model.add(Flatten(name="flat"))
    model.add(Dense(12, activation='linear', name="dense1", trainable=False))
    
    # new dense output
    model.add(Dense(number_of_outputs, activation='linear', name="dense2"))
    
    return model

In [ ]:
model = create_model(145, 1)

In [ ]:
model.summary()#line_length=None, positions=None, print_fn=None)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# set the weights that we want (tested and works)

for i in range(len(model.layers)-1):
    pretrained_layer_weights = pretrained_model.layers[i].get_weights()
    model.layers[i].set_weights(pretrained_layer_weights)

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# load in the data
df = pd.read_csv("data/raw/hidra.tsv", sep="\t", header=None)

# make separate dataframes for chloroplast data and mitochondrion data
df_chloro = df[df[0] == "NC_016734.1"][[3,8]]  
df_mito = df[df[0] == "NC_008285.1"][[3,8]]

In [ ]:
X_chloro = np.array([get_ohe(sqnc) for sqnc in df_chloro[3]])  # create proper representation
y_chloro = np.array(df_chloro[8].tolist())

In [ ]:
X_chloro.shape, y_chloro.shape

((30544, 145, 4), (30544,))

In [ ]:
# create train/test/val sequentially

idx = int(X_chloro.shape[0]/10)

X_test = X_chloro[:idx]
y_test = y_chloro[:idx]

X_val = X_chloro[idx:2*idx]
y_val = y_chloro[idx:2*idx]

X_train = X_chloro[2*idx:]
y_train = y_chloro[2*idx:]

In [ ]:
# train new model

model.compile(optimizer='adam',
            loss="mean_squared_error")

history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))

Train on 24436 samples, validate on 3054 samples
Epoch 1/10
24436/24436 [==============================] - 17s 690us/step - loss: 0.2781 - val_loss: 0.2057
Epoch 2/10
24436/24436 [==============================] - 16s 669us/step - loss: 0.1782 - val_loss: 0.1628
Epoch 3/10
24436/24436 [==============================] - 15s 613us/step - loss: 0.1758 - val_loss: 0.1637
Epoch 4/10
24436/24436 [==============================] - 16s 658us/step - loss: 0.1760 - val_loss: 0.1648
Epoch 5/10
24436/24436 [==============================] - 17s 710us/step - loss: 0.1754 - val_loss: 0.1658
Epoch 6/10
24436/24436 [==============================] - 22s 884us/step - loss: 0.1754 - val_loss: 0.1655
Epoch 7/10
24436/24436 [==============================] - 21s 865us/step - loss: 0.1750 - val_loss: 0.1601
Epoch 8/10
24436/24436 [==============================] - 18s 756us/step - loss: 0.1748 - val_loss: 0.1595
Epoch 9/10
24436/24436 [==============================] - 24s 990us/step - loss: 0.1742 - val_l

In [ ]:
# R^2
print("Train", r2_score(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train 0.025520088173720423
Val 0.14674706940165294
Test -1.993770569559346


In [ ]:
# Spearman
print("Train", spearmanr(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.5221309934494401, pvalue=0.0)
Val SpearmanrResult(correlation=0.4450641571374498, pvalue=1.6324941583393092e-148)
Test SpearmanrResult(correlation=0.3936982832739136, pvalue=8.878059168448573e-114)


---
### Chloroplast w/ Replaced Layer

In [ ]:
def create_model(input_sequence_length, number_of_outputs):
    model = Sequential()

    model.add(Conv1D(120, 5, activation='relu', input_shape=(input_sequence_length, 4), name="1DConv_1", trainable=False))
    model.add(BatchNormalization(name="batchNorm1", trainable=False))
    model.add(Dropout(0.1, name="drop1"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=False))
    model.add(BatchNormalization(name="batchNorm2", trainable=False))
    model.add(Dropout(0.1, name="drop2"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=False))
    model.add(BatchNormalization(name="batchNorm3", trainable=False))
    model.add(Dropout(0.1, name="drop3"))

    model.add(Flatten(name="flat"))
#     model.add(Dense(12, activation='linear', name="dense1", trainable=False))
    
    model.add(Dense(number_of_outputs, activation='linear', name="dense2"))
    
    return model

In [ ]:
model = create_model(145, 1)

In [ ]:
model.summary()#line_length=None, positions=None, print_fn=None)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# set the weights that we want (tested and works)

for i in range(len(model.layers)-1):
    pretrained_layer_weights = pretrained_model.layers[i].get_weights()
    model.layers[i].set_weights(pretrained_layer_weights)

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# load in the data
df = pd.read_csv("data/raw/hidra.tsv", sep="\t", header=None)

# TODO: standardize data

df_chloro = df[df[0] == "NC_016734.1"][[3,8]]
df_mito = df[df[0] == "NC_008285.1"][[3,8]]

In [ ]:
X_chloro = np.array([get_ohe(sqnc) for sqnc in df_chloro[3]])
y_chloro = np.array(df_chloro[8].tolist())

In [ ]:
X_chloro.shape, y_chloro.shape

((30544, 145, 4), (30544,))

In [ ]:
idx = int(X_chloro.shape[0]/10)

X_test = X_chloro[:idx]
y_test = y_chloro[:idx]

X_val = X_chloro[idx:2*idx]
y_val = y_chloro[idx:2*idx]

X_train = X_chloro[2*idx:]
y_train = y_chloro[2*idx:]

In [ ]:
# train new model

model.compile(optimizer='adam',
            loss="mean_squared_error")

history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))

Train on 24436 samples, validate on 3054 samples
Epoch 1/10
24436/24436 [==============================] - 17s 699us/step - loss: 0.1739 - val_loss: 0.1695
Epoch 2/10
24436/24436 [==============================] - 16s 652us/step - loss: 0.1608 - val_loss: 0.1766
Epoch 3/10
24436/24436 [==============================] - 16s 647us/step - loss: 0.1547 - val_loss: 0.1796
Epoch 4/10
24436/24436 [==============================] - 16s 673us/step - loss: 0.1491 - val_loss: 0.1842
Epoch 5/10
24436/24436 [==============================] - 16s 653us/step - loss: 0.1459 - val_loss: 0.1834
Epoch 6/10
24436/24436 [==============================] - 16s 660us/step - loss: 0.1421 - val_loss: 0.1919
Epoch 7/10
24436/24436 [==============================] - 16s 654us/step - loss: 0.1397 - val_loss: 0.1907
Epoch 8/10
24436/24436 [==============================] - 17s 693us/step - loss: 0.1381 - val_loss: 0.2037
Epoch 9/10
24436/24436 [==============================] - 16s 657us/step - loss: 0.1362 - val_l

In [ ]:
# R^2
print("Train", r2_score(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train 0.305479799583078
Val -0.04177023622108811
Test -2.0856430092333516


In [ ]:
# Spearman
print("Train", spearmanr(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.7238977464110093, pvalue=0.0)
Val SpearmanrResult(correlation=0.29193534866384485, pvalue=4.5447566658393835e-61)
Test SpearmanrResult(correlation=0.3175957726749138, pvalue=1.5550787606399824e-72)


## C3, C4, C5

### C3 Chloroplast: L1 = F, L2 = F, L3 = F

In [ ]:
def create_model(input_sequence_length, number_of_outputs):
    model = Sequential()

    model.add(Conv1D(120, 5, activation='relu', input_shape=(input_sequence_length, 4), name="1DConv_1", trainable=True))
    model.add(BatchNormalization(name="batchNorm1", trainable=True))
    model.add(Dropout(0.1, name="drop1"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=True))
    model.add(BatchNormalization(name="batchNorm2", trainable=True))
    model.add(Dropout(0.1, name="drop2"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=True))
    model.add(BatchNormalization(name="batchNorm3", trainable=True))
    model.add(Dropout(0.1, name="drop3"))

    model.add(Flatten(name="flat"))
#     model.add(Dense(12, activation='linear', name="dense1", trainable=False))
    
    model.add(Dense(number_of_outputs, activation='linear', name="dense2"))
    
    return model

In [ ]:
model = create_model(145, 1)

In [ ]:
model.summary()#line_length=None, positions=None, print_fn=None)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# set the weights that we want (tested and works)

for i in range(len(model.layers)-1):
    print(model.layers[i])
    pretrained_layer_weights = pretrained_model.layers[i].get_weights()
    model.layers[i].set_weights(pretrained_layer_weights)

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# load in the data
df = pd.read_csv("data/raw/hidra.tsv", sep="\t", header=None)

# TODO: standardize data

df_chloro = df[df[0] == "NC_016734.1"][[3,8]]
# df_mito = df[df[0] == "NC_008285.1"][[3,8]]

In [ ]:
X_chloro = np.array([get_ohe(sqnc) for sqnc in df_chloro[3]])
y_chloro = np.array(df_chloro[8].tolist())

In [ ]:
X_chloro.shape, y_chloro.shape

((30544, 145, 4), (30544,))

In [ ]:
idx = int(X_chloro.shape[0]/10)

X_test = X_chloro[:idx]
y_test = y_chloro[:idx]

X_val = X_chloro[idx:2*idx]
y_val = y_chloro[idx:2*idx]

X_train = X_chloro[2*idx:]
y_train = y_chloro[2*idx:]

In [ ]:
# train new model

model.compile(optimizer='adam',
            loss="mean_squared_error")

history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))

Train on 24436 samples, validate on 3054 samples
Epoch 1/10
24436/24436 [==============================] - 51s 2ms/step - loss: 0.0878 - val_loss: 0.3216
Epoch 2/10
24436/24436 [==============================] - 63s 3ms/step - loss: 0.0260 - val_loss: 0.2809
Epoch 3/10
24436/24436 [==============================] - 56s 2ms/step - loss: 0.0187 - val_loss: 0.2868
Epoch 4/10
24436/24436 [==============================] - 56s 2ms/step - loss: 0.0158 - val_loss: 0.3136
Epoch 5/10
24436/24436 [==============================] - 51s 2ms/step - loss: 0.0146 - val_loss: 0.2621
Epoch 6/10
24436/24436 [==============================] - 55s 2ms/step - loss: 0.0131 - val_loss: 0.2414
Epoch 7/10
24436/24436 [==============================] - 51s 2ms/step - loss: 0.0122 - val_loss: 0.2963
Epoch 8/10
24436/24436 [==============================] - 55s 2ms/step - loss: 0.0115 - val_loss: 0.2558
Epoch 9/10
24436/24436 [==============================] - 53s 2ms/step - loss: 0.0108 - val_loss: 0.3033
Epoch 

In [ ]:
# R^2
print("Train", r2_score(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train 0.9799335241758428
Val -0.4995104242444577
Test -5.636248290009296


In [ ]:
# Spearman
print("Train", spearmanr(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9922069131053126, pvalue=0.0)
Val SpearmanrResult(correlation=0.33976088308434715, pvalue=2.1632926427376007e-83)
Test SpearmanrResult(correlation=0.19787366531644104, pvalue=2.451821229188447e-28)


---
### C4 Chloroplast: L1 = Z, L2 = Z, L3 = F

In [ ]:
def create_model(input_sequence_length, number_of_outputs):
    model = Sequential()

    model.add(Conv1D(120, 5, activation='relu', input_shape=(input_sequence_length, 4), name="1DConv_1", trainable=False))
    model.add(BatchNormalization(name="batchNorm1", trainable=False))
    model.add(Dropout(0.1, name="drop1"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=False))
    model.add(BatchNormalization(name="batchNorm2", trainable=False))
    model.add(Dropout(0.1, name="drop2"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=True))
    model.add(BatchNormalization(name="batchNorm3", trainable=True))
    model.add(Dropout(0.1, name="drop3"))

    model.add(Flatten(name="flat"))
#     model.add(Dense(12, activation='linear', name="dense1", trainable=False))
    
    model.add(Dense(number_of_outputs, activation='linear', name="dense2"))
    
    return model

In [ ]:
model = create_model(145, 1)

In [ ]:
model.summary()#line_length=None, positions=None, print_fn=None)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# set the weights that we want (tested and works)

for i in range(len(model.layers)-1):
    print(model.layers[i])
    pretrained_layer_weights = pretrained_model.layers[i].get_weights()
    model.layers[i].set_weights(pretrained_layer_weights)

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# load in the data
df = pd.read_csv("data/raw/hidra.tsv", sep="\t", header=None)

# TODO: standardize data

df_chloro = df[df[0] == "NC_016734.1"][[3,8]]
df_mito = df[df[0] == "NC_008285.1"][[3,8]]

In [ ]:
X_chloro = np.array([get_ohe(sqnc) for sqnc in df_chloro[3]])
y_chloro = np.array(df_chloro[8].tolist())

In [ ]:
X_chloro.shape, y_chloro.shape

((30544, 145, 4), (30544,))

In [ ]:
idx = int(X_chloro.shape[0]/10)

X_test = X_chloro[:idx]
y_test = y_chloro[:idx]

X_val = X_chloro[idx:2*idx]
y_val = y_chloro[idx:2*idx]

X_train = X_chloro[2*idx:]
y_train = y_chloro[2*idx:]

In [ ]:
# train new model

model.compile(optimizer='adam',
            loss="mean_squared_error")

history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))

Train on 24436 samples, validate on 3054 samples
Epoch 1/10
24436/24436 [==============================] - 24s 988us/step - loss: 0.1248 - val_loss: 0.2167
Epoch 2/10
24436/24436 [==============================] - 23s 929us/step - loss: 0.0418 - val_loss: 0.2129
Epoch 3/10
24436/24436 [==============================] - 22s 895us/step - loss: 0.0295 - val_loss: 0.1983
Epoch 4/10
24436/24436 [==============================] - 27s 1ms/step - loss: 0.0249 - val_loss: 0.2264
Epoch 5/10
24436/24436 [==============================] - 27s 1ms/step - loss: 0.0218 - val_loss: 0.2264
Epoch 6/10
24436/24436 [==============================] - 26s 1ms/step - loss: 0.0207 - val_loss: 0.2380
Epoch 7/10
24436/24436 [==============================] - 24s 994us/step - loss: 0.0198 - val_loss: 0.2107
Epoch 8/10
24436/24436 [==============================] - 23s 956us/step - loss: 0.0184 - val_loss: 0.2223
Epoch 9/10
24436/24436 [==============================] - 24s 979us/step - loss: 0.0173 - val_loss: 0

In [ ]:
# R^2
print("Train", r2_score(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train 0.8871313556493472
Val -0.12134065629052238
Test -3.498811411763822


In [ ]:
# Spearman
print("Train", spearmanr(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9863863021497022, pvalue=0.0)
Val SpearmanrResult(correlation=0.30289889295843436, pvalue=7.943610210692869e-66)
Test SpearmanrResult(correlation=0.19439020557687545, pvalue=2.1821821628929946e-27)


---
### C3 Chloroplast: L1 = Z, L2 = Z, L3 = R

In [ ]:
def create_model(input_sequence_length, number_of_outputs):
    model = Sequential()

    model.add(Conv1D(120, 5, activation='relu', input_shape=(input_sequence_length, 4), name="1DConv_1", trainable=False))
    model.add(BatchNormalization(name="batchNorm1", trainable=False))
    model.add(Dropout(0.1, name="drop1"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=False))
    model.add(BatchNormalization(name="batchNorm2", trainable=False))
    model.add(Dropout(0.1, name="drop2"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=True))
    model.add(BatchNormalization(name="batchNorm3", trainable=True))
    model.add(Dropout(0.1, name="drop3"))

    model.add(Flatten(name="flat"))
#     model.add(Dense(12, activation='linear', name="dense1", trainable=False))
    
    model.add(Dense(number_of_outputs, activation='linear', name="dense2"))
    
    return model

In [ ]:
model = create_model(145, 1)

In [ ]:
model.summary()#line_length=None, positions=None, print_fn=None)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# set the weights that we want (tested and works)

for i in range(6):
    print(model.layers[i])
    pretrained_layer_weights = pretrained_model.layers[i].get_weights()
    model.layers[i].set_weights(pretrained_layer_weights)

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# load in the data
df = pd.read_csv("data/raw/hidra.tsv", sep="\t", header=None)

# TODO: standardize data

df_chloro = df[df[0] == "NC_016734.1"][[3,8]]
df_mito = df[df[0] == "NC_008285.1"][[3,8]]

In [ ]:
X_chloro = np.array([get_ohe(sqnc) for sqnc in df_chloro[3]])
y_chloro = np.array(df_chloro[8].tolist())

In [ ]:
X_chloro.shape, y_chloro.shape

((30544, 145, 4), (30544,))

In [ ]:
idx = int(X_chloro.shape[0]/10)

X_test = X_chloro[:idx]
y_test = y_chloro[:idx]

X_val = X_chloro[idx:2*idx]
y_val = y_chloro[idx:2*idx]

X_train = X_chloro[2*idx:]
y_train = y_chloro[2*idx:]

In [ ]:
# train new model

model.compile(optimizer='adam',
            loss="mean_squared_error")

history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))

Train on 24436 samples, validate on 3054 samples
Epoch 1/10
24436/24436 [==============================] - 25s 1ms/step - loss: 1.7100 - val_loss: 0.7388
Epoch 2/10
24436/24436 [==============================] - 24s 980us/step - loss: 0.5105 - val_loss: 0.3701
Epoch 3/10
24436/24436 [==============================] - 25s 1ms/step - loss: 0.2220 - val_loss: 0.2876
Epoch 4/10
24436/24436 [==============================] - 24s 994us/step - loss: 0.1234 - val_loss: 0.2508
Epoch 5/10
24436/24436 [==============================] - 25s 1ms/step - loss: 0.0748 - val_loss: 0.2276
Epoch 6/10
24436/24436 [==============================] - 25s 1ms/step - loss: 0.0502 - val_loss: 0.2125
Epoch 7/10
24436/24436 [==============================] - 24s 980us/step - loss: 0.0346 - val_loss: 0.2002
Epoch 8/10
24436/24436 [==============================] - 24s 986us/step - loss: 0.0274 - val_loss: 0.2014
Epoch 9/10
24436/24436 [==============================] - 26s 1ms/step - loss: 0.0233 - val_loss: 0.218

In [ ]:
# R^2
print("Train", r2_score(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train 0.9358380801416339
Val -0.254193614432878
Test -3.7695818839165547


In [ ]:
# Spearman
print("Train", spearmanr(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9797530730630867, pvalue=0.0)
Val SpearmanrResult(correlation=0.27975972285828044, pvalue=5.008215429418851e-56)
Test SpearmanrResult(correlation=0.20483495827101586, pvalue=2.7433648175799054e-30)


### C3 Mitochondrion: L1 = F, L2 = F, L3 = F

In [ ]:
def create_model(input_sequence_length, number_of_outputs):
    model = Sequential()

    model.add(Conv1D(120, 5, activation='relu', input_shape=(input_sequence_length, 4), name="1DConv_1", trainable=True))
    model.add(BatchNormalization(name="batchNorm1", trainable=True))
    model.add(Dropout(0.1, name="drop1"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=True))
    model.add(BatchNormalization(name="batchNorm2", trainable=True))
    model.add(Dropout(0.1, name="drop2"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=True))
    model.add(BatchNormalization(name="batchNorm3", trainable=True))
    model.add(Dropout(0.1, name="drop3"))

    model.add(Flatten(name="flat"))
#     model.add(Dense(12, activation='linear', name="dense1", trainable=False))
    
    model.add(Dense(number_of_outputs, activation='linear', name="dense2"))
    
    return model

In [ ]:
model = create_model(145, 1)

In [ ]:
model.summary()#line_length=None, positions=None, print_fn=None)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# set the weights that we want (tested and works)

for i in range(len(model.layers)-1):
    print(model.layers[i])
    pretrained_layer_weights = pretrained_model.layers[i].get_weights()
    model.layers[i].set_weights(pretrained_layer_weights)

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# load in the data
df = pd.read_csv("data/raw/hidra.tsv", sep="\t", header=None)

# TODO: standardize data

# df_chloro = df[df[0] == "NC_016734.1"][[3,8]]
df_chloro = df[df[0] == "NC_008285.1"][[3,8]]

In [ ]:
X_chloro = np.array([get_ohe(sqnc) for sqnc in df_chloro[3]])
y_chloro = np.array(df_chloro[8].tolist())

In [ ]:
X_chloro.shape, y_chloro.shape

((44342, 145, 4), (44342,))

In [ ]:
idx = int(X_chloro.shape[0]/10)

X_test = X_chloro[:idx]
y_test = y_chloro[:idx]

X_val = X_chloro[idx:2*idx]
y_val = y_chloro[idx:2*idx]

X_train = X_chloro[2*idx:]
y_train = y_chloro[2*idx:]

In [ ]:
# train new model

model.compile(optimizer='adam',
            loss="mean_squared_error")

history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))

Train on 35474 samples, validate on 4434 samples
Epoch 1/10
35474/35474 [==============================] - 70s 2ms/step - loss: 0.1130 - val_loss: 0.3406
Epoch 2/10
35474/35474 [==============================] - 69s 2ms/step - loss: 0.0306 - val_loss: 0.2838
Epoch 3/10
35474/35474 [==============================] - 73s 2ms/step - loss: 0.0231 - val_loss: 0.2740
Epoch 4/10
35474/35474 [==============================] - 77s 2ms/step - loss: 0.0199 - val_loss: 0.2953
Epoch 5/10
35474/35474 [==============================] - 72s 2ms/step - loss: 0.0179 - val_loss: 0.3156
Epoch 6/10
35474/35474 [==============================] - 73s 2ms/step - loss: 0.0166 - val_loss: 0.2642
Epoch 7/10
35474/35474 [==============================] - 68s 2ms/step - loss: 0.0154 - val_loss: 0.2752
Epoch 8/10
35474/35474 [==============================] - 69s 2ms/step - loss: 0.0142 - val_loss: 0.2628
Epoch 9/10
35474/35474 [==============================] - 74s 2ms/step - loss: 0.0136 - val_loss: 0.2466
Epoch 

In [ ]:
# R^2
print("Train", r2_score(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train 0.960030166856412
Val -0.2553586740139038
Test -0.0474876591929847


In [ ]:
# Spearman
print("Train", spearmanr(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9831030515052255, pvalue=0.0)
Val SpearmanrResult(correlation=0.05786917496219279, pvalue=0.00011546788033856874)
Test SpearmanrResult(correlation=0.12038642119929258, pvalue=8.742740315960421e-16)


---
### C4 Mitochondrion: L1 = Z, L2 = Z, L3 = F

In [ ]:
def create_model(input_sequence_length, number_of_outputs):
    model = Sequential()

    model.add(Conv1D(120, 5, activation='relu', input_shape=(input_sequence_length, 4), name="1DConv_1", trainable=False))
    model.add(BatchNormalization(name="batchNorm1", trainable=False))
    model.add(Dropout(0.1, name="drop1"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=False))
    model.add(BatchNormalization(name="batchNorm2", trainable=False))
    model.add(Dropout(0.1, name="drop2"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=True))
    model.add(BatchNormalization(name="batchNorm3", trainable=True))
    model.add(Dropout(0.1, name="drop3"))

    model.add(Flatten(name="flat"))
#     model.add(Dense(12, activation='linear', name="dense1", trainable=False))
    
    model.add(Dense(number_of_outputs, activation='linear', name="dense2"))
    
    return model

In [ ]:
model = create_model(145, 1)

In [ ]:
model.summary()#line_length=None, positions=None, print_fn=None)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# set the weights that we want (tested and works)

for i in range(len(model.layers)-1):
    print(model.layers[i])
    pretrained_layer_weights = pretrained_model.layers[i].get_weights()
    model.layers[i].set_weights(pretrained_layer_weights)

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# load in the data
df = pd.read_csv("data/raw/hidra.tsv", sep="\t", header=None)

# TODO: standardize data

# df_chloro = df[df[0] == "NC_016734.1"][[3,8]]
df_chloro = df[df[0] == "NC_008285.1"][[3,8]]

In [ ]:
X_chloro = np.array([get_ohe(sqnc) for sqnc in df_chloro[3]])
y_chloro = np.array(df_chloro[8].tolist())

In [ ]:
X_chloro.shape, y_chloro.shape

((44342, 145, 4), (44342,))

In [ ]:
idx = int(X_chloro.shape[0]/10)

X_test = X_chloro[:idx]
y_test = y_chloro[:idx]

X_val = X_chloro[idx:2*idx]
y_val = y_chloro[idx:2*idx]

X_train = X_chloro[2*idx:]
y_train = y_chloro[2*idx:]

In [ ]:
# train new model

model.compile(optimizer='adam',
            loss="mean_squared_error")

history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))

Train on 35474 samples, validate on 4434 samples
Epoch 1/10
35474/35474 [==============================] - 36s 1ms/step - loss: 0.1564 - val_loss: 0.3014
Epoch 2/10
35474/35474 [==============================] - 33s 920us/step - loss: 0.0523 - val_loss: 0.2876
Epoch 3/10
35474/35474 [==============================] - 31s 868us/step - loss: 0.0365 - val_loss: 0.2821
Epoch 4/10
35474/35474 [==============================] - 31s 874us/step - loss: 0.0311 - val_loss: 0.2724
Epoch 5/10
35474/35474 [==============================] - 31s 882us/step - loss: 0.0284 - val_loss: 0.2862
Epoch 6/10
35474/35474 [==============================] - 32s 913us/step - loss: 0.0257 - val_loss: 0.2942
Epoch 7/10
35474/35474 [==============================] - 32s 907us/step - loss: 0.0241 - val_loss: 0.2979
Epoch 8/10
35474/35474 [==============================] - 31s 875us/step - loss: 0.0230 - val_loss: 0.2942
Epoch 9/10
35474/35474 [==============================] - 31s 877us/step - loss: 0.0221 - val_los

In [ ]:
# R^2
print("Train", r2_score(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train 0.9415857396748929
Val -0.22931888773526987
Test -0.20688776875565962


In [ ]:
# Spearman
print("Train", spearmanr(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9741109089254318, pvalue=0.0)
Val SpearmanrResult(correlation=0.07419364169473011, pvalue=7.581153582165634e-07)
Test SpearmanrResult(correlation=-0.021965243001119066, pvalue=0.14363383603781982)


---
### C3 Mitochondrion: L1 = Z, L2 = Z, L3 = R

In [ ]:
def create_model(input_sequence_length, number_of_outputs):
    model = Sequential()

    model.add(Conv1D(120, 5, activation='relu', input_shape=(input_sequence_length, 4), name="1DConv_1", trainable=False))
    model.add(BatchNormalization(name="batchNorm1", trainable=False))
    model.add(Dropout(0.1, name="drop1"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=False))
    model.add(BatchNormalization(name="batchNorm2", trainable=False))
    model.add(Dropout(0.1, name="drop2"))

    model.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=True))
    model.add(BatchNormalization(name="batchNorm3", trainable=True))
    model.add(Dropout(0.1, name="drop3"))

    model.add(Flatten(name="flat"))
#     model.add(Dense(12, activation='linear', name="dense1", trainable=False))
    
    model.add(Dense(number_of_outputs, activation='linear', name="dense2"))
    
    return model

In [ ]:
model = create_model(145, 1)

In [ ]:
model.summary()#line_length=None, positions=None, print_fn=None)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# set the weights that we want (tested and works)

for i in range(6):
    print(model.layers[i])
    pretrained_layer_weights = pretrained_model.layers[i].get_weights()
    model.layers[i].set_weights(pretrained_layer_weights)

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          72120     
__________

In [ ]:
# load in the data
df = pd.read_csv("data/raw/hidra.tsv", sep="\t", header=None)

# TODO: standardize data

# df_chloro = df[df[0] == "NC_016734.1"][[3,8]]
df_chloro = df[df[0] == "NC_008285.1"][[3,8]]

In [ ]:
X_chloro = np.array([get_ohe(sqnc) for sqnc in df_chloro[3]])
y_chloro = np.array(df_chloro[8].tolist())

In [ ]:
X_chloro.shape, y_chloro.shape

((44342, 145, 4), (44342,))

In [ ]:
idx = int(X_chloro.shape[0]/10)

X_test = X_chloro[:idx]
y_test = y_chloro[:idx]

X_val = X_chloro[idx:2*idx]
y_val = y_chloro[idx:2*idx]

X_train = X_chloro[2*idx:]
y_train = y_chloro[2*idx:]

In [ ]:
# train new model

model.compile(optimizer='adam',
            loss="mean_squared_error")

history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_val, y_val))

Train on 35474 samples, validate on 4434 samples
Epoch 1/10
35474/35474 [==============================] - 33s 926us/step - loss: 1.4642 - val_loss: 0.5751
Epoch 2/10
35474/35474 [==============================] - 31s 877us/step - loss: 0.3483 - val_loss: 0.3729
Epoch 3/10
35474/35474 [==============================] - 33s 934us/step - loss: 0.1423 - val_loss: 0.3170
Epoch 4/10
35474/35474 [==============================] - 36s 1ms/step - loss: 0.0732 - val_loss: 0.3007
Epoch 5/10
35474/35474 [==============================] - 31s 880us/step - loss: 0.0470 - val_loss: 0.2919
Epoch 6/10
35474/35474 [==============================] - 32s 910us/step - loss: 0.0349 - val_loss: 0.2935
Epoch 7/10
35474/35474 [==============================] - 33s 926us/step - loss: 0.0295 - val_loss: 0.2660
Epoch 8/10
35474/35474 [==============================] - 33s 937us/step - loss: 0.0265 - val_loss: 0.3031
Epoch 9/10
35474/35474 [==============================] - 31s 871us/step - loss: 0.0241 - val_los

In [ ]:
# R^2
print("Train", r2_score(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train 0.9489978382498858
Val -0.30910819657873745
Test -0.15048862029150722


In [ ]:
# Spearman
print("Train", spearmanr(y_train, model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.96592136407161, pvalue=0.0)
Val SpearmanrResult(correlation=0.012712260445004118, pvalue=0.3973947908291495)
Test SpearmanrResult(correlation=0.021540618220178786, pvalue=0.15153949320643711)
